# R Coding Assignment \#3: Conditional Probability and Independence

### Sheetal Parikh
EN.605.631.81<br>
September 20, 2020
***
***

## Problem 1
As flu season is approaching, we all have a chance of getting the flu.  Let P(flu) be the number of flu cases last year, divided by the number of people potentially exposed to the flu.  Let P(vaccine) be the number of people that got a flu vaccination last year, divided by the number of people potentially exposed to the flu.

The confusion matrix for this is notionally:

                                        Vaccine           No Vaccine
                    Flu                   11                 1,329           

                    Healthy             5,211               13,247      

a.) What is the probability of getting the flu?

b.) What is the probability of getting the flu, given a person is vaccinated?

c.) What is the probability of getting the flu, given a person is not vaccinated?

d.) What is the probability someone was vaccinated, given they got the flu?

First we can summarize the given information as a table:

In [17]:
#Displaying table of given information

#install.packages("data.table", dependencies=TRUE)

library(data.table)

#creating table
table<-data.table(Cases = c("Flu","Healthy"),
                  Vaccine = c(11,5211),
                  No_Vaccine = c(1329,13247))

#adding row of totals
table1<-rbind(table,data.table(Cases='Total',Vaccine=sum(table$Vaccine),No_Vaccine=sum(table$No_Vaccine)))

#adding column of totals
table1$Total=rowSums(table1[,c(-1)])

#printing the resulting table
table1

Cases,Vaccine,No_Vaccine,Total
Flu,11,1329,1340
Healthy,5211,13247,18458
Total,5222,14576,19798


a.) Probability of getting the flu:

We are given that the probability of getting the flu, which we call P(flu), is the total number of flu cases divided by the total number of people potentially exposed to the flu.  The total number of flu cases as shown in the table is 1,340 and the total number of people potentially exposed is 19,798 giving us P(flu) = 1,340/19,798 = 0.06768 or 6.768%.

In [18]:
#calculating P(flu)
p_flu <- round(table1[1,Total]/table1[3, Total], digits=5)
p_flu

[1] 0.06768

b.) Probability of getting the flu, given a person is vaccinated:

To calculate the P(flu|vaccinated) we need to calculate (Flu and Vaccinated)/Total number of vaccinations.  From the table we can se that there is a total of 5,222 vaccinations.  We can also see that the number of people that got the flu and were also vaccinated is 11.  Therefore, P(flu/vaccinated) = 11/5,222 = 0.00211

In [19]:
#calculating P(flu/vaccinated)
p_flu.vaccinated <- round(table1[1,Vaccine]/table1[3,Vaccine], digits=5)
p_flu.vaccinated

[1] 0.00211

c.) Probability of getting the flu, given a person is not vaccinated:

To calculate P(flu|not_vaccinated), we need to determine the number of (Flu and Not_Vaccinated)/Not_Vaccinated.  From the table we can see that the total number of non_vaccinations is 14,576. The total number of people that got the flu who didn't get a vaccination is 1,329.  Therefore, P(flu|not_vaccinated) = 1,329/,14,576 = 0.09118

In [20]:
#calculating P(flu/not_vaccinated)
p_flu.notvaccinated <- round(table1[1,No_Vaccine]/table1[3,No_Vaccine], digits=5)
p_flu.notvaccinated

[1] 0.09118

d.) Probability someone was vaccinated, given they got the flu:

To calculate P(vaccinated|flu), we need to determine the number of (Vaccinated and Flu)/Flu cases.  From the table we can see that the total number of people who got the vaccinated and also got the flu is 11.  The total number of flu cases is 1,340.  Threfore, P(vaccinated|flu) = 11/1,340 = 0.00821.

In [21]:
#calculating P(vaccinated/flu)
p_vaccinated.flu <- round(table1[1,Vaccine]/table1[1,Total], digits=5)
p_vaccinated.flu

[1] 0.00821

## Problem 2
Is getting the flu independent of vaccination?  Support your conclusion statistically (e.g. prove it!)

As per the definition of independence we know that if two events E and F are said to be independent then, P(EF) = P(E)P(F).  If (PE) is not equal to P(E)P(F) then the two events are said to be dependent.  If we want to know whether getting the flu is independent of vaccinations then we need to determine the value of P(flu and Vaccinations), P(flu), and P(vaccinations).  From problem 1 we already determined that P(flu) = 0.06768.  We can calculate P(vaccinations) = (number of vaccinations)/total number of people potentially exposed).  From the table we can see that P(vaccinations) = 5,222/19,798 = 0.26376.  Therefore, P(flu)P(vaccinations) = 0.06768 x 0.26376 = 0.01785.  Lastly, P(flu and vaccinations) = (number of people who got both a flu and were vaccinated)/(number of people potentially exposed) = 11/19,798 = 0.000556. Since 0.01785 is not equal to 0.0005556 we can determine that getting the flu is dependent on vaccination.  Getting the flu is not independent of vaccinations.

In [22]:
#from Problem 1 we know P(flu) is the variable p_flu
cat("P(flu):", p_flu)
cat("\n")

#calculating P(vaccination) 
p_vaccination <- round(table1[3,Vaccine]/table1[3, Total], digits=5)
cat("P(vaccination):", p_vaccination)
cat("\n")

p_vaccinationp_flu = round(p_flu*p_vaccination, digits = 5) 
cat("P(flu)*P(Vaccination):", p_vaccinationp_flu)
cat("\n")

#calculating P(flu and vaccination)
p_fluandvaccination <- round(table1[1,Vaccine]/table1[3, Total], digits=5)
cat("P(flu and vaccination):", p_fluandvaccination)
cat("\n")

#compare if p_vaccination*p_flu = p_fluandvaccination
cat("\n")
independent <- setequal(p_vaccinationp_flu, p_fluandvaccination)
cat("Is getting the flu independent of vaccinations?: ", independent)

P(flu): 0.06768
P(vaccination): 0.26376
P(flu)*P(Vaccination): 0.01785
P(flu and vaccination): 0.00056

Is getting the flu independent of vaccinations?:  FALSE

## Problem 3
We introduce a flu test that is positive 86% of the time it is tested on someone with the flu and is negative 95% of the time it is tested on a completely healthy person.  Further assume that 2% of the population has the flu. Usually people are the most likely to infect others when they are pre-symptomatic (before they show symptoms).  The state governor, in an attempt to reduce the spread of the disease, passes a bill to increase flu testing and encourages people to be tested. 

a.) What is the probability someone has the flu if they test positive? 

b.) What is the probability someone will test positive if they don't have the flu?

We can summarize the given information through through simulating who has the flu:

In [23]:
#taking random sample of 1,000,000 people w/ replacement who either have or do not have the flu
#as given in the problem, 2% of the population has the flu and 100%-2% = 98% does not have the flu
flu <- sample(c('No','Yes'), size=1000000, replace=TRUE, prob=c(0.98,0.02))

#creating dummy variable and sample population of 1,000,000
test <- rep(NA, 1000000) 

#of those in the sample above, 95% of the people who do not have the disease will test negative and 5% will falsely test positive
test[flu=='No'] <- sample(c('Neg','Pos'), size=sum(flu=='No'), replace=TRUE, prob=c(0.95,0.05))

#of those in the sample above, 14% of the people who have the disease will falsely test negative and 86% will test postive
test[flu=='Yes'] <- sample(c('Neg','Pos'), size=sum(flu=='Yes'), replace=TRUE, prob=c(0.14, 0.86))


a.) Probability someone has the flu if they test positive?

We can use Bayes' theorem to calculate the probability, P(Flu|+) = P(+|Flu)P(Flu)/P(+).  From the given information we know that P(Flu) = 0.02.  We can calculate P(+|Flu) as (the probability of having the flu) times (the probability of a positive test if tested on someone with the flu) which is: P(+|Flu) = (0.02 x 0.86) = 0.0172.  P(+) is the probability of just having a positive test for both people that have and do not have the disease. We can calculate P(+) as the following: P(+) = P(flu)P(+ if flu) + P(not flu)P(+ if no flu) = (0.02 x 0.86) + (0.98 x 0.05) = 0.0662.  Therefore, P(Flu|+) = 0.0172/0.0662 = 0.2598, meaning there is approximately a 26% chance of the flu test having a true positive result.  The probability can also be calculated using a random sample as shown below:

In [24]:
#calculating the probability of a true positive using the random sample from above (line 9)
round(mean(flu[test=='Pos']=='Yes'), digits = 5)


[1] 0.25997

b.) Probability someone will test positive if they don't have the flu?

We can use Bayes' theorem to calculate the probability, P(+|no flu) = P(no flu|+)P(no flu)/P(no flu). From the given information we know that P(no Flu) = 0.98. We can calculate P(no flu|+) as not having the flu times a positive test if tested on someone who doesn't have the flu which is: P(no flu|+) = (0.98 x 0.05) = 0.049. P(no flu) is the probability of not having the flu which we know is 0.98. Therefore, P(+|no flu) = 0.049/0.98 = 0.05, meaning there is about a 5% chance of the flu test having a false positive result(which is what we expect based on the given information). The probability can also be calculated using a random sample as shown below:

In [25]:
#calculating the probability of a false positive using the random sample from above (line 9)
round(mean(test[flu=='No']=='Pos'), digits = 5)

[1] 0.04982

## Problem 4
What conclusions can you draw about flu vaccinations and testing?  What recommendations would you offer the state governor?  Make sure to support your recommendations with valid statistics.



Conclusions about Testing:

As we can see from Problem 3, the probability of having a positive result if you actually have the flu, meaning a true positive result, is only roughly 26%.  This is much lower than initially expected from a test that is fairly accurate having an 86% chance of a positive result in people with the flu.  This discrepancy in the probability can be attributed to the very low rate of actually having the flu.  Since only a small portion of the population actually get the flu, it makes sense that only a small portion will test positive.

For example if we take the calculation from Problem 3, and change the probability of getting the flu from 2% to 10%, you can see the probability of getting a true positive result drastically get larger from 26% to 66%.  

In [26]:
#taking random sample of 1,000,000 people w/ replacement who either have or do not have the flu
#If 10% of the population has the flu and 100%-10% = 90% does not have the flu
flu2 <- sample(c('No','Yes'), size=1000000, replace=TRUE, prob=c(0.90,0.10))

#creating dummy variable and sample population of 1,000,000
test2 <- rep(NA, 1000000) 

#of those in the sample above, 95% of the people who do not have the disease will test negative and 5% will falsely test positive
test2[flu2=='No'] <- sample(c('Neg','Pos'), size=sum(flu2=='No'), replace=TRUE, prob=c(0.95,0.05))

#of those in the sample above, 14% of the people who have the disease will falsely test negative and 86% will test postive
test2[flu2=='Yes'] <- sample(c('Neg','Pos'), size=sum(flu2=='Yes'), replace=TRUE, prob=c(0.14, 0.86))

#calculating the probability of a true positive using the random sample 
truepos<-round(mean(flu2[test2=='Pos']=='Yes'), digits = 5)
cat("P(truepositive) = P(+|Flu):", truepos)
cat("\n")

P(truepositive) = P(+|Flu): 0.65709


We can also use the true positive and false positive probabilies from problem 3 to calculate a ratio of the probability of a false positive to a true positive result to give us: P(false positive/true positive) = 0.05/0.26 = 0.19.  Therefore, this can mean that it is more likely to have a positive diagnosis that is true (and the flu test works better) when the ratio of the number of people who have the disease to the number of people who do not have the disease is at least 0.19.  This is much larger to the actual given rate of the disease of 2%, demonstrating that we are likely to have inaccurate results. However, if there is a higher number of people that get the flu as already shown above, the ratio of false positives to true positives also improves:

In [27]:
#Our example using a higher rate of the flu, 10% Flu and 90% No Flu
#calculating the probability of a true positive using the random sample
falsepos<-round(mean(test2[flu2=='No']=='Pos'), digits = 5)
cat("P(falsepositive) = P(+|NoFlu):", falsepos)
cat("\n")

#ratio of false positive to true positive
ratio<-round(falsepos/truepos, digits = 5)
cat("Ratio of (false positive/true positive):", ratio)
cat("\n")

P(falsepositive) = P(+|NoFlu): 0.04986
Ratio of (false positive/true positive): 0.07588


Conclusions about Flu Vaccinations:

From Problem 2 we determined how the number of flu cases is dependent on the number of flu vaccinations and saw just how important flu vaccinations are to whether or not someone actually gets the flu. Due to this dependency, we can assume that the number of flu cases would go down with an increased number of vaccinations.  This makes sense with the  herd mentality theory of how more people getting a vaccine would help protect one person as well as the community.  When people are vaccinated it will be harder for someone else to get infected with the flu making it more difficult for the flu to spread accross the community.

From Problem 1 we also saw just how signficiant getting the flu vaccine is to the overall outcome of getting or not getting the flue.  For example, if the probability of getting the flu is about 6.8%, the likelihood of getting the flu is less than 1% (actually 0.211%) if you just get the vaccine.  However, the likelihood of getting the flu is much higher at 9% if you do not get vaccinated.  We can also look at the people who got the flu and see how many of those people had actually gotten the vaccine.  As per Problem 1, part d, we calculated how less than 1% of the people who had gotten the flu were vaccinated.  That means that 1-0.00821 or roughly 99% of those people who had gotten the flu were people who never got the vaccine (calculation also shown below).  We are assuming that the numbers reported are all true positives and no false positives.  However, even if any of the numbers were false positives, it is still very apparent how getting the flu vaccine overwhelmingly lessens the chances of someone getting the flu. 

In [28]:
#calculating P(not vaccinated/flu) = 99%
p_notvaccinated.flu <- round(table1[1,No_Vaccine]/table1[1,Total], digits=5)
p_notvaccinated.flu

[1] 0.99179

Recommendations for the Governor:

Since actually testing for whether you have the flu is affected by the prevalence of the flu, I don't believe that that testing alone should be used as a way to control the spread. I would urge the governor to encourage people to get vaccinated as soon as vaccines are available rather than just getting tested.  His bill should include educating the public and promoting the importance of getting flu vaccinations rather than just for flu testing. Although testing for the virus is important for understanding the spread and potentially determining the length of the flu season, it doesn't directly impact the spread of the virus, whereas getting vaccinated does directly impact the spread. If most of the population would get vaccinated as easly as possible, it would also reduce the chance of pre-symptomatic spreading of the flu. As it is less likely to get the flu if someone is vaccinated, we could assume that less people would go to the physician to get tested, which may also reduce the number of tests performed that result in false positives.  Overall, the governor should focus on explaining the important of the flu vaccine and encouraging the people to get vaccinated as soon as the flu vaccines become available rather than only encouraging people to just get tested for the flu.

References:

Problem 1: 

https://www.datasciencemadesimple.com/row-wise-sum-row-sum-in-r-dataframe-2/

https://stackoverflow.com/questions/21872262/r-how-to-get-a-single-element-in-data-table

https://astrostatistics.psu.edu/su07/R/library/base/html/colSums.html


Problem 2:

https://www.geeksforgeeks.org/check-if-two-objects-are-equal-in-r-programming-setequal-function/#:~:text=setequal()%20function%20in%20R,Objects%20are%20equal%20or%20not.


Problem 3: 

https://medium.com/district-data-labs/conditional-probability-with-r-5544c6886621

https://stats.stackexchange.com/questions/15141/how-to-create-a-dataset-with-conditional-probability

Problem 4:

https://www.vaccines.gov/basics/work/protection

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3153801